In [ ]:
import sys
import random
import pickle

import pandas as pd
import numpy as np
import json

import math
import datetime

from sklearn import preprocessing, cross_validation, svm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVC

from sklearn.externals import joblib
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


import matplotlib.pyplot as plt


In [ ]:
# teller_df = train_df[train_df['name']=='teller']
pred_lst = []
total = 0
true_t = 0
for idx, name in enumerate(sbm_in[:]):

    teller_df = train_df[train_df['name']==name]
    teller_df = teller_df.set_index('date')

    teller_df['hl_pct'] = (teller_df['high'] - teller_df['low'])/(teller_df['low']*100)
    teller_df['pct_chng'] = (teller_df['close'] - teller_df['open'])/(teller_df['open']*100)
    


    # 예측 컬럼
    forecast_col = 'close'
    # 예측 기간 하루
    forecast_out = int(1)
    # forecast_out에 맞춰 label을 변경
    teller_df['label'] = teller_df[forecast_col].shift(-forecast_out)
    # 방향 예측을 위한 전처리
    teller_df['direction'] = teller_df['label'] >= teller_df['close']

    # 과거 close
    teller_df['before_0'] = teller_df[forecast_col].shift(forecast_out)
    teller_df['before_1'] = teller_df[forecast_col].shift(forecast_out+1)
    teller_df['before_2'] = teller_df[forecast_col].shift(forecast_out+2)
    
    

    
    teller_df = teller_df[3:]
    
    teller_df['direction_0'] = teller_df['before_0'] >= teller_df['before_1']
    teller_df['direction_1'] = teller_df['before_1'] >= teller_df['before_2']
    
    # X, y 'hl_pct','pct_chng',
    X = teller_df[['close','hl_pct','pct_chng','before_0','before_1']]
    X = preprocessing.scale(X)
    X_forecast_out = X[-forecast_out:]
    X = X[:-forecast_out]

    y = np.array(teller_df['label'])
    y = y[:-forecast_out]
    y_d = np.array(teller_df['direction'])
    y_d = y_d[:-forecast_out]

    if idx % 50 == 0:
        print(idx,'counts name: ',name, '. Length of y: ', len(y))
    
    #model = SVR(kernel='linear',C=0.01).fit(X,y)
    model = LinearRegression().fit(X,y)
    #model_d = LogisticRegression(C=0.1).fit(X,y_d)
    model_d = xgboost.XGBClassifier(n_estimators=100,max_depth=7).fit(X,y_d)
    y_pred = model.predict(X_forecast_out)
    y_d_pred = model_d.predict(X_forecast_out)

    if y_d_pred:
        val = '+'
    else:
        val = '-'
   # val = '+'
    val += ' ' + str(float(y_pred))

    pred_lst.append(str(val))